# 载入数据集

In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd
from IPython.display import clear_output
import keras

Using TensorFlow backend.


In [2]:
keras.__version__

'2.2.4'

In [3]:
df = pd.read_csv('../data/labels.csv')
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
n = len(df)
breed = set(df['breed'])
n_class = len(breed)
class_to_num = dict(zip(breed, range(n_class)))
num_to_class = dict(zip(range(n_class), breed))

In [5]:
width = 299
y = np.zeros((n, n_class), dtype=np.uint8)
for i in tqdm(range(int(n / 2))):
    y[i][class_to_num[df['breed'][i]]] = 1

100%|██████████| 5111/5111 [00:00<00:00, 38406.29it/s]


# 导出特征

In [6]:
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input

In [7]:
def get_features(MODEL):
    cnn_model = MODEL(include_top=False, input_shape=(width, width, 3), weights='imagenet')
    
    inputs = Input((width, width, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = cnn_model(x)
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs, x)
    
    batch_size = 32
    num_batches = n // batch_size
    last_batch_size = n - batch_size * num_batches
    num_batches = num_batches + 1 if last_batch_size > 0 else num_batches
    
    #print('Num batches:', num_batches, 'Last_batch size:', last_batch_size)
    
    k = 0
    features = np.zeros((n, 2048))
    for i in tqdm(range(num_batches)):
        size = batch_size if i != num_batches - 1 else last_batch_size
        X = np.zeros((batch_size, width, width, 3)) if i != num_batches - 1 else np.zeros((last_batch_size, width, width, 3))        
        for j in range(size):
            X[j] = cv2.resize(cv2.imread('../data/train/%s.jpg' % df['id'][k]), (width, width))
            k = k + 1
        predictons = cnn_model.predict(X, batch_size=batch_size, verbose=0)

        features[i * batch_size: i * batch_size + size, :] = predictons

        clear_output(wait=True)
        
    return features

In [ ]:
inception_features = get_features(InceptionV3)
np.save('./inception_features.npy', inception_features)

In [ ]:
xception_features = get_features(Xception)
np.save('./xception_features.npy', xception_features)

In [8]:
inception_features = np.load('./inception_features.npy')
xception_features = np.load('./xception_features.npy')
features = np.concatenate([inception_features, xception_features], axis=-1)

# 训练模型

In [9]:
inputs = Input(features.shape[1:])
x = inputs
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax')(x)
model = Model(inputs, x)

model.compile(
    optimizer=tf.train.AdamOptimizer(),
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

h = model.fit(
    features, 
    y, 
    batch_size=128, 
    epochs=10, 
    validation_split=0.1,
    shuffle=True,
)

Train on 9199 samples, validate on 1023 samples
Epoch 1/10
9199/9199 [==============================] - 1s 159us/step - loss: 1.2060 - acc: 0.3078 - val_loss: 0.0000e+00 - val_acc: 0.0078
Epoch 2/10
9199/9199 [==============================] - 1s 115us/step - loss: 0.3384 - acc: 0.4712 - val_loss: 0.0000e+00 - val_acc: 0.0029
Epoch 3/10
9199/9199 [==============================] - 1s 111us/step - loss: 0.2403 - acc: 0.4944 - val_loss: 0.0000e+00 - val_acc: 0.0049
Epoch 4/10
9199/9199 [==============================] - 1s 112us/step - loss: 0.1863 - acc: 0.5094 - val_loss: 0.0000e+00 - val_acc: 0.0049
Epoch 5/10
9199/9199 [==============================] - 1s 112us/step - loss: 0.1473 - acc: 0.5241 - val_loss: 0.0000e+00 - val_acc: 0.0059
Epoch 6/10
9199/9199 [==============================] - 1s 124us/step - loss: 0.1251 - acc: 0.5281 - val_loss: 0.0000e+00 - val_acc: 0.0049
Epoch 7/10
9199/9199 [==============================] - 1s 115us/step - loss: 0.0997 - acc: 0.5382 - val_loss: 0

# 可视化模型

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

# 可视化训练过程

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')

# 载入测试集

In [ ]:
df2 = pd.read_csv('sample_submission.csv')

In [ ]:
n_test = len(df2)
X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
for i in tqdm(range(n_test)):
    X_test[i] = cv2.resize(cv2.imread('test/%s.jpg' % df2['id'][i]), (width, width))

# 导出测试集特征

In [ ]:
inception_features = get_features(InceptionV3, X_test)
xception_features = get_features(Xception, X_test)
features_test = np.concatenate([inception_features, xception_features], axis=-1)

# 预测测试集并输出结果

In [ ]:
y_pred = model.predict(features_test, batch_size=128)

In [ ]:
for b in breed:
    df2[b] = y_pred[:,class_to_num[b]]

In [ ]:
df2.to_csv('pred.csv', index=None)